In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import odds_datasets


def load_sims(dataset, batch_size, strategy, data_copies):
    save_dir = f"results/{dataset}x{data_copies}/{strategy}/batch_size_{batch_size}"
    files = [f"{save_dir}/{f}" for f in os.listdir(save_dir)]
    sim = np.stack([np.load(f)["avp_test"] for f in files], axis=-1)
    return sim


def plot_sim(
    dataset,
    data_copies=1,
    batch_sizes=[1, 2, 5, 10],
    strategies=["worstcase", "average", "bestcase", "independent"],
):
    if dataset == "legend":
        for batch_size, color in zip(batch_sizes, plt.cm.tab10.colors):
            plt.plot([], [], label=f"batch size {batch_size}", color=color)

        for strategy, linestyle in zip(strategies, ["-", "--", "-.", ":"]):
            plt.plot([], [], label=strategy, linestyle=linestyle, color="black")
        plt.legend()
        return
    for strategy, linestyle in zip(strategies, ["-", "--", "-.", ":"]):
        for batch_size, color in zip(batch_sizes, plt.cm.tab10.colors):
            sim = load_sims(dataset, batch_size, strategy, data_copies)
            x = np.arange(len(sim)) / len(sim) * 100
            value = sim.mean(-1)
            lb = value - sim.std(-1)
            ub = value + sim.std(-1)
            plt.plot(x, value, color=color, linestyle=linestyle)
            plt.fill_between(x, lb, ub, alpha=0.1, color=color)
            plt.grid(True)
            plt.xlim(0, 10)


plt.figure(figsize=(9, 18))
for i, dataset in enumerate(odds_datasets.datasets_names):
    plt.subplot(6, 3, i + 1)
    plt.title(f"{dataset}")
    try:
        plot_sim(dataset, data_copies=5)
    except:
        pass
plt.subplot(6, 3, 18)
plot_sim("legend")
plt.axis("off")
plt.tight_layout()
plt.savefig("results/plot.pdf")
plt.show()